In [ ]:
!pip install pycaret[full]
!pip install opendatasets
!pip install pandas

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
import pycaret
pycaret.__version__

'3.3.2'

In [ ]:
!kaggle datasets download -d julienjta/nyc-taxi-traffic
!unzip nyc-taxi-traffic.zip


Dataset URL: https://www.kaggle.com/datasets/julienjta/nyc-taxi-traffic
License(s): CC0-1.0
  0% 0.00/80.1k [00:00<?, ?B/s]
100% 80.1k/80.1k [00:00<00:00, 49.6MB/s]
Archive:  nyc-taxi-traffic.zip
  inflating: dataset.csv             


In [ ]:
import pandas as pd
df = pd.read_csv('dataset.csv')
df.head()

,Unnamed: 0,timestamp,value
0,0,2014-07-01 00:00:00,10844
1,1,2014-07-01 00:30:00,8127
2,2,2014-07-01 01:00:00,6210
3,3,2014-07-01 01:30:00,4656
4,4,2014-07-01 02:00:00,3820


In [ ]:
# import pycaret anomaly and init setup
from pycaret.anomaly import *
s = setup(df, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(10320, 3)"
2,Transformed data shape,"(10320, 10322)"
3,Numeric features,2
4,Categorical features,1
5,Preprocess,True
6,Imputation type,simple
7,Numeric imputation,mean
8,Categorical imputation,mode
9,Maximum one-hot encoding,-1


In [ ]:
# train iforest model
iforest = create_model('iforest')
iforest

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0)

In [ ]:
iforest_anomalies = assign_model(iforest)
iforest_anomalies

,Unnamed: 0,timestamp,value,Anomaly,Anomaly_Score
0,0,2014-07-01 00:00:00,10844,0,-0.001601
1,1,2014-07-01 00:30:00,8127,0,-0.001601
2,2,2014-07-01 01:00:00,6210,1,0.001474
3,3,2014-07-01 01:30:00,4656,0,-0.000967
4,4,2014-07-01 02:00:00,3820,0,-0.000967
...,...,...,...,...,...
10315,10315,2015-01-31 21:30:00,24670,0,-0.000538
10316,10316,2015-01-31 22:00:00,25721,0,-0.000538
10317,10317,2015-01-31 22:30:00,27309,0,-0.000538
10318,10318,2015-01-31 23:00:00,26591,0,-0.000538


In [ ]:
# tsne plot anomalies
plot_model(iforest, plot = 'tsne')

In [13]:
# predict on test set
iforest_pred = predict_model(iforest, data=df)
iforest_pred

,Unnamed: 0,timestamp_2014-07-01 00:00:00,timestamp_2014-07-01 00:30:00,timestamp_2014-07-01 01:00:00,timestamp_2014-07-01 01:30:00,timestamp_2014-07-01 02:00:00,timestamp_2014-07-01 02:30:00,timestamp_2014-07-01 03:00:00,timestamp_2014-07-01 03:30:00,timestamp_2014-07-01 04:00:00,...,timestamp_2015-01-31 20:30:00,timestamp_2015-01-31 21:00:00,timestamp_2015-01-31 21:30:00,timestamp_2015-01-31 22:00:00,timestamp_2015-01-31 22:30:00,timestamp_2015-01-31 23:00:00,timestamp_2015-01-31 23:30:00,value,Anomaly,Anomaly_Score
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10844.0,0,-0.001601
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8127.0,0,-0.001601
2,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6210.0,1,0.001474
3,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4656.0,0,-0.000967
4,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3820.0,0,-0.000967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10315,10315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,24670.0,0,-0.000538
10316,10316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,25721.0,0,-0.000538
10317,10317.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,27309.0,0,-0.000538
10318,10318.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,26591.0,0,-0.000538


In [14]:
# save pipeline
save_model(iforest, 'iforest_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Unnamed: 0', 'value'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['timestamp'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['timestamp'],
                                     transformer=OneHotEncoder(cols=['timestamp'],
                                                               handle_missing='return_nan',
                                                               use_cat_names=True))),
                 ('trained_model',
                  IForest(behaviour='new', bootstrap=False, contamination=0.05,
     max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
     random_state=123, verbose=0))]),
 'ifore

In [15]:
# load pipeline
loaded_iforest_pipeline = load_model('iforest_pipeline')
loaded_iforest_pipeline

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Unnamed: 0', 'value'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['timestamp'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 TransformerWrapper(include=['timestamp'],
                                    transformer=OneHotEncoder(cols=['timestamp'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('trained_model',
                 IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0))])